In [1]:
import shutil
import subprocess
import os
import numpy as np
import pandas as pd

In [7]:
#Path to the bat file that compiles the VPSC program
bat_file = r'C:\Users\frear\OneDrive - NTNU\HØST 2024\Prosjektoppgave\KODE\VPSC_code-main\VPSC_code-main\compileVPSC8.bat'

def compileVPSC():
    try:
        # Compile the vpsc8.exe from the .bat file and capture the output
        result = subprocess.run([bat_file],shell=True,check=True,capture_output=True,text=True)
        print("Output:\n", result.stdout)
    except subprocess.CalledProcessError as e:
        # In case of an error
        print(f"Error occurred: {e}")
        print("Error output:\n", e.stderr)

#Provide the path for the Fortran program as well as providing the current working directory where the input files are located
exe_path = r'C:\Users\frear\OneDrive - NTNU\HØST 2024\Prosjektoppgave\KODE\VPSC_code-main\VPSC_code-main\vpsc8.exe'
VPSC_directory = r'C:\Users\frear\OneDrive - NTNU\HØST 2024\Prosjektoppgave\KODE\VPSC_code-main\VPSC_code-main'

def executeVPSC():
    try:
        result = subprocess.run([exe_path], capture_output=True, text=True,cwd=VPSC_directory)
        if result.returncode != 0:  
            raise Exception('VPSC failed')
    except subprocess.CalledProcessError as e:
        # In case of an error
        print(f"Error occurred: {e}")
        print("Error output:\n", e.stderr)

def run_VPSC():
    compileVPSC()
    executeVPSC()

run_VPSC()

Output:
 


In [33]:
#Path to the output file from the VPSC code TEX_PH1.OUT
texturePath = r'C:\Users\frear\OneDrive - NTNU\HØST 2024\Prosjektoppgave\KODE\VPSC_code-main\VPSC_code-main\TEX_PH1.OUT'

"""VPSC code generates a single file with grain orientations stored at intervals specified in the vpsc8.in file.
The following cell block reads the output file from the VPSC code and splits it into 20 different crystal texture files.
The crystal texture files are then to be used for input into yield surface calculations"""

def generate_skip_indices(total_rows, interval=1004, skip_count=4):
    
    """Function that generates indices for which rows to skip in the TEX_PH1.OUT file. standard is 1000 grains, so 4 rows will be skipped every 1000 orientations"""

    skip_indices = []
    for i in range(0, total_rows, interval):
        skip_indices.extend(range(i, min(i + skip_count, total_rows)))  
    return skip_indices

class texture:
    def __init__(self):
        self.textureFiles = [] 

def make_texture_list(num,strain_interval=0.1, strainmax=2):

    """Function to make a list of paths to textures for input into yield surface calculator"""
    skipIndices = generate_skip_indices(int(strainmax/strain_interval*1004))
    texturesEveryTen = pd.read_csv(texturePath,sep='\s+',skiprows=skipIndices,names=['a','b','c','d'])

    texturesEveryTenFiltered = texturesEveryTen.to_numpy().reshape(-1,1000,4)
    
    textureFiles  = []
    outputFileNames  = []

    for i in range(texturesEveryTenFiltered.shape[0]):
            
        filename = f'texture_{num}_{i}.tex'
        newFileNamePCYS = f'PCYS_{num}_{i}.OUT'

        file_path = os.path.join(VPSC_directory, filename)
        
        textureFiles.append((file_path,f'textures_{num}\{filename}'))
        outputFileNames.append(newFileNamePCYS)
    
    return textureFiles,outputFileNames

def make_texture_files(process,num,strainmax=2, strain_interval=0.1):

    """Function that reads the output from the VPSC code file TEX_PH1.OUT and separates it into individual texture files"""
    skipIndices = generate_skip_indices(int(strainmax/strain_interval*1004))
    texturesEveryTen = pd.read_csv(texturePath,sep='\s+',skiprows=skipIndices,names=['a','b','c','d'])

    texturesEveryTenFiltered = texturesEveryTen.to_numpy().reshape(-1,1000,4)

    header = [
        "dummy", 
        "dummy", 
        f"texture generated by {process}", 
        "B   1000"
    ]
    textureDirectory  = os.path.join(VPSC_directory,f'textures_{num}')

    if os.path.exists(textureDirectory):
        raise Exception('Folder already exists, Check if you are about to overwrite your data')
        
    elif not os.path.exists(textureDirectory):
            os.makedirs(textureDirectory)
    
    textureFiles  = []
    outputFileNames  = []

    for i in range(texturesEveryTenFiltered.shape[0]):

        filename = f'texture_{num}_{i}.tex'
        newFileNamePCYS = f'PCYS_{num}_{i}.OUT'

# Define the full file path, including the folder and file name
        file_path = os.path.join(textureDirectory, filename)
        
        textureFiles.append((file_path,f'textures_{num}\{filename}'))
        outputFileNames.append(newFileNamePCYS)
        
        subarray = texturesEveryTenFiltered[i]
        df = pd.DataFrame(subarray)

        with open(file_path, 'w') as file:
            for line in header:
                file.write(line + '\n')
        
        with open(file_path, 'a') as file:
            # Format each row to align columns with consistent spacing
            for row in df.itertuples(index=False):
                formatted_row = "{:9.2f} {:9.2f} {:9.2f} {:10.7f}".format(*row)
                file.write(formatted_row + '\n')



In [34]:
#Path to the input file vpsc8.in
input_file_path = r'C:\Users\frear\OneDrive - NTNU\HØST 2024\Prosjektoppgave\KODE\VPSC_code-main\VPSC_code-main\vpsc8.in' 

outputFileNamePCYS = 'PCYS.OUT'
yieldSurfacePath = os.path.join(VPSC_directory,outputFileNamePCYS)


def Yield_surface_calculator(textureFiles, outputFileNames,L_num,start=0):
    
    PCYS_folder = os.path.join(VPSC_directory,f'PCYS_{L_num}')
    if os.path.exists(PCYS_folder):
        raise Exception('Folder already exists, Check if you are about to overwrite your data')
    elif not os.path.exists(PCYS_folder):
            os.makedirs(PCYS_folder)

    for i,texture_file in enumerate(textureFiles[start:]):
        
            # Update the line with the current texture filename
        with open(input_file_path, 'r') as file:
            lines = file.readlines()
        
        lines[8] = f"{texture_file[1]}\n"

        if lines[-2].strip() == "0":
            # Remove the last two lines and replace them with -2
            lines = lines[:-2]  # Remove last two lines
            lines.append("-2\n")
            
        with open(input_file_path, 'w') as file:
            file.writelines(lines)
        
        run_VPSC()
        
        newYieldSurfacePath = os.path.join(PCYS_folder,outputFileNames[i+start])
        
        shutil.copy(yieldSurfacePath,newYieldSurfacePath )
        

In [35]:
import f2py_yldfit as fyf
class VPSC:
    def __init__(self,exe,bat,input):
        self.exePath = exe
        self.batPath = bat
        self.inputPath = input


In [ ]:
velocityGradientsPath = r'C:\Users\frear\OneDrive - NTNU\HØST 2024\Prosjektoppgave\KODE\VPSC_code-main\VPSC_code-main\VelocityGradients'
originalFile = r'C:\Users\frear\OneDrive - NTNU\HØST 2024\Prosjektoppgave\KODE\VPSC_code-main\VPSC_code-main\ex02_FCC\rolling.3'


def create_L_files(num_of_Ls):

    thetas = np.linspace(0,np.pi/4,num_of_Ls)

    if not os.path.exists(velocityGradientsPath):
        os.makedirs(velocityGradientsPath)
            
    else: raise Exception('Folder already exists')
    
    for i,theta in enumerate(thetas):
        
        LPath = os.path.join(velocityGradientsPath,f'L_{i}')
        shutil.copy(originalFile,LPath)

        diag_values = np.array([np.cos(theta),np.sin(theta),-np.cos(theta)-np.sin(theta)])
        vonMisesNorm = np.sqrt(2/3)*np.linalg.norm(diag_values)
        diag_values = diag_values/vonMisesNorm

        with open(LPath, 'r') as file:
            lines = file.readlines()
        
        lines[6] = f"    {diag_values[0]:.2f}    0.       0.          udot    |    vel.grad\n"
        # Assuming line 6 (index 5) contains '0.      1.0     0.'
        lines[7] = f"    0.      {diag_values[1]:.2f}     0.                  |\n"
        # Assuming line 7 (index 6) contains '0.      0.     -1.0'
        lines[8] = f"    0.      0.      {diag_values[2]:.2f}                |\n"
        
        with open(LPath, 'w') as file:
            file.writelines(lines)

def Texture_varying_L_calculator(num_of_Ls,start=0, new_Ls=True):
    
    if new_Ls:
        create_L_files(num_of_Ls)
    else: pass
    velocityGradientFiles = os.listdir(velocityGradientsPath)

    for i,L_file in enumerate(velocityGradientFiles[start:]):
        print(L_file)
            # Update the line with the current texture filename
        with open(input_file_path, 'r') as file:
            lines = file.readlines()
        
        lines[8]  = 'Rand1000.tex\n'
        if lines[-1].strip() == "-2":
            lines[-1] = f"{0}\n"
            lines.append(f"VelocityGradients\{L_file}\n")
        else:
            lines[-1] = f"VelocityGradients\{L_file}\n"

        with open(input_file_path, 'w') as file:
            file.writelines(lines)   

        run_VPSC()
        make_texture_files(L_file, i)
        textureFiles,outputFileNames = make_texture_list(i)
        print(textureFiles,outputFileNames)
        Yield_surface_calculator(textureFiles,outputFileNames,i)

Texture_varying_L_calculator(10,new_Ls=False)

L_0
Output:
 
[('C:\\Users\\frear\\OneDrive - NTNU\\HØST 2024\\Prosjektoppgave\\KODE\\VPSC_code-main\\VPSC_code-main\\texture_0_0.tex', 'textures_0\\texture_0_0.tex'), ('C:\\Users\\frear\\OneDrive - NTNU\\HØST 2024\\Prosjektoppgave\\KODE\\VPSC_code-main\\VPSC_code-main\\texture_0_1.tex', 'textures_0\\texture_0_1.tex'), ('C:\\Users\\frear\\OneDrive - NTNU\\HØST 2024\\Prosjektoppgave\\KODE\\VPSC_code-main\\VPSC_code-main\\texture_0_2.tex', 'textures_0\\texture_0_2.tex'), ('C:\\Users\\frear\\OneDrive - NTNU\\HØST 2024\\Prosjektoppgave\\KODE\\VPSC_code-main\\VPSC_code-main\\texture_0_3.tex', 'textures_0\\texture_0_3.tex'), ('C:\\Users\\frear\\OneDrive - NTNU\\HØST 2024\\Prosjektoppgave\\KODE\\VPSC_code-main\\VPSC_code-main\\texture_0_4.tex', 'textures_0\\texture_0_4.tex'), ('C:\\Users\\frear\\OneDrive - NTNU\\HØST 2024\\Prosjektoppgave\\KODE\\VPSC_code-main\\VPSC_code-main\\texture_0_5.tex', 'textures_0\\texture_0_5.tex'), ('C:\\Users\\frear\\OneDrive - NTNU\\HØST 2024\\Prosjektoppgave\\KO

In [8]:
TSigmaInverse = 1/np.sqrt(6)*np.array([[np.sqrt(2),-1,-np.sqrt(3),0 ,0 ,0 ],
                                      [np.sqrt(2) ,-1 ,np.sqrt(3) ,0 ,0 ,0 ],
                                      [np.sqrt(2), 2, 0, 0, 0, 0],
                                      [0, 0, 0, np.sqrt(3), 0, 0],
                                      [0, 0, 0, 0 ,np.sqrt(3), 0],
                                      [0, 0, 0, 0, 0, np.sqrt(3)]])
print(np.matrix(TSigmaInverse*np.sqrt(6)))

[[ 1.41421356 -1.         -1.73205081  0.          0.          0.        ]
 [ 1.41421356 -1.          1.73205081  0.          0.          0.        ]
 [ 1.41421356  2.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          1.73205081  0.          0.        ]
 [ 0.          0.          0.          0.          1.73205081  0.        ]
 [ 0.          0.          0.          0.          0.          1.73205081]]


In [1]:
b1 = 1/np.sqrt(2) * np.array([[-1, 0, 0], [0, 1, 0], [0, 0, 0]])
b2 = 1/np.sqrt(6) * np.array([[-1, 0, 0], [0, -1, 0], [0, 0, 2]])
b3 = 1/np.sqrt(2) * np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]])
b4 = 1/np.sqrt(2) * np.array([[0, 0, 1], [0, 0, 0], [1, 0, 0]])
b5 = 1/np.sqrt(2) * np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]])

# Stack the basis matrices along a new dimension (shape: (5, 3, 3))
basis_matrices = np.stack([b1, b2, b3, b4, b5], axis=0)

def from_vpsc(S):
    S_expanded = S[:, :, np.newaxis, np.newaxis]
    print(S_expanded.shape)  # Shape (7680, 5, 1, 1)
    stress_tensors = np.sum(S_expanded * basis_matrices, axis=1)
    return stress_tensors

NameError: name 'np' is not defined

In [10]:
""" compileVPSC()
run_VPSC()
make_texture_files("L_1",1) """
textureFiles,outputFilenames = make_texture_list(1) 
Yield_surface_calculator(textureFiles,outputFilenames)

Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
Output:
 
Exit Code: 0
